In [132]:
import numpy as np
import pandas as pd
#from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split

train_url = "C:\Users\Vishnutej\Desktop\pml_train.csv"
train_set = pd.read_csv(train_url)
#print(train.head())
test_url = "C:\Users\Vishnutej\Desktop\pml_test_features.csv"
test_set =pd.read_csv(test_url)
#print(train["id"].value_counts())
train,test = train_test_split(train,test_size=0.1,random_state=42)
#All the features excluding the labels
features = [x for x in train.columns if x not in ['id','loss']]
#print(features)

#Categorical Features
cat_features = [x for x in train.select_dtypes(include=['object']).columns if x not in ['id','loss']]
#print(cat_features)

#Numerical Features
num_features = [x for x in train.select_dtypes(exclude=['object']).columns if x not in ['id','loss']]
#print(num_features)


print ('Mean of loss Variable'+' '+'is'+' '+ str(np.mean(train["loss"])))
print ('Median of loss Variable'+' '+'is'+' '+ str(np.median(train["loss"])))
print ('Standard Deviation of loss Variable'+' '+'is'+' '+ str(np.std(train["loss"])))

Mean of loss Variable is 3036.19545173
Median of loss Variable is 2113.78
Standard Deviation of loss Variable is 2930.01187561


In [133]:
#Plotting the Loss 

import statsmodels.api as sm
from matplotlib import pyplot as plt
fig=sm.qqplot(train["loss"])
plt.show()

In [134]:
#Plotting Log(loss)

from scipy.stats import norm, lognorm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

train['log_loss'] = np.log(train['loss'])

# Fit the log(loss)
(mu, sigma) = norm.fit(train['log_loss'])

# Histogram
n, bins, patches = plt.hist(train['log_loss'], 60, normed=1, facecolor='black', alpha=0.5)

y = mlab.normpdf( bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

#Graph
plt.xlabel('Ln(loss)')
plt.ylabel('Probability')
plt.title(r'$\mathrm{Histogram\ of\ Ln(Loss):}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
plt.grid(True)

plt.show()

In [135]:
#Convert categorical variables

for each in cat_features:
    train[each]=pd.factorize(train[each], sort=True)[0]
    test[each]=pd.factorize(test[each],sort=True)[0]
    test_set[each]=pd.factorize(test_set[each],sort=True)[0]

In [136]:
#Random forest regressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae
Predictors= train.ix[:,0:130]
Predictors_test= test.ix[:,0:130]

MAE=[]
Model_Name=[]
#rf = RandomForestRegressor(n_estimator=100) 
rfs = [RandomForestRegressor(),RandomForestRegressor(n_estimators=100)]
for rf in rfs: 
    Model=rf.fit(Predictors,train['log_loss'])
    Prediction= np.exp(Model.predict(Predictors_test))
    eva = mae(test['loss'],Prediction)
    #print(str(eva))
    MAE.append(eva)
    Name=rf.__class__.__name__
    Model_Name.append(Name)
    print('Accuracy of'+ ' '+Name+' '+'is'+' '+str(eva))

Accuracy of RandomForestRegressor is 1277.86652447
Accuracy of RandomForestRegressor is 1227.36306005


In [137]:
#Lasso

from sklearn.linear_model import Lasso

#las = [Lasso(alpha=0.1),Lasso(alpha=1.0)]
lass=[Lasso(alpha=1.0),Lasso(alpha=0.1)]

for las in lass:
    Model=las.fit(Predictors,train['log_loss'])
    Prediction= np.exp(Model.predict(Predictors_test))
    eva = mae(test['loss'],Prediction)
    MAE.append(eva)
    Name=las.__class__.__name__
    Model_Name.append(Name)
    print('Accuracy of'+ ' '+Name+' '+'is'+' '+str(eva))
print(MAE)

Accuracy of Lasso is 1810.71939472
Accuracy of Lasso is 1492.80461683
[1277.8665244738083, 1227.363060045765, 1810.7193947150668, 1492.8046168294381]


In [138]:
#Comparison of the models.
Index = [1,2,3,4]
plt.bar(Index,MAE)
plt.xticks(Index, Model_Name,rotation=45)
plt.ylabel('MAE')
plt.xlabel('Model')
plt.title('MAE of Models')
plt.show()

ImportError: No module named xgboost

In [164]:
features =Predictors.columns
train['log_loss'] =np.log(train['loss'])
final_reg = RandomForestRegressor(n_estimators=500)
for each in features:
    train[each]=pd.factorize(train[each], sort=True)[0]
    #test_set[each]=pd.factorize(test_set[each],sort=True)[0]
#del test_set["id"];
train_array=np.array(train[features])
mod=final_reg.fit(Predictors,train['log_loss'])
test_array=np.array(test_set)
final = np.exp(mod.predict(test_array))
print("here")
submission = pd.read_csv('C:\Users\Vishnutej\Desktop\pml_example_submission.csv')
submission.iloc[:, 1] = final
submission.to_csv('C:\Users\Vishnutej\Desktop\\file.csv', index=None)
print(submission)
print("Done")

here
           id         loss
0      131822  3933.117822
1      131823  3689.457017
2      131824  5144.664605
3      131825  3618.703123
4      131826  4605.443110
5      131827  3892.943937
6      131828  3835.883803
7      131829  3629.304423
8      131830  8943.294749
9      131831  3995.404012
10     131832  4114.210877
11     131833  4156.898973
12     131834  3604.965323
13     131835  5081.743458
14     131836  3994.577412
15     131837  2438.796133
16     131838  4290.043713
17     131839  3686.471823
18     131840  4255.396426
19     131841  4486.837602
20     131842  2419.824662
21     131843  2437.295763
22     131844  4937.930294
23     131845  2970.094912
24     131846  3583.217210
25     131847  3253.377430
26     131848  2671.008927
27     131849  5211.988534
28     131850  4283.896597
29     131851  4365.855717
...       ...          ...
56466  188288  1711.397125
56467  188289  2448.650256
56468  188290  3885.714837
56469  188291  4043.219208
56470  188292  3710.332